In [ ]:
import pdfplumber

def skaityti_pdf(kelias_iki_pdf):
    tekstas = ""
    page = 0
    with pdfplumber.open(kelias_iki_pdf) as pdf:
        for puslapis in pdf.pages:
            page+=1
            puslapio_tekstas = puslapis.extract_text()
            if puslapio_tekstas:
                tekstas  += puslapio_tekstas + "**********"
            if page > 400:
                break
    return tekstas


lt_text = skaityti_pdf('baltaragio_malunas.pdf')
lt_pages = lt_text.replace('\n', ' ').replace(',', '').replace('- ', '').split("**********")[3:]

en_text = skaityti_pdf('SherlockHolmesComplete.pdf')
en_pages = en_text.replace('\n', ' ').replace(',', '').replace('- ', '').split("**********")[11:]

en_pages = en_pages[:len(lt_pages)]

lt_labels, en_labels = [], []
for page in lt_pages:
    lt_labels.append("lt")
    en_labels.append("en")	


from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

lt_en_texts = lt_pages + en_pages
lt_en_labels = lt_labels + en_labels

X_train, X_test, y_train, y_test = train_test_split(lt_en_texts, lt_en_labels, test_size=0.2, random_state=0)

model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(accuracy_score(y_pred, y_test))
print("Classification Report:\n", classification_report(y_test, y_pred, zero_division=0))